In [ ]:
"""
Блок 1. Оптимизация с помощью индекса
"""

import libs.db_connection

# Интересует конкретный пользователь
sql1 = "explain (analyze) select * from block11 where user_id = 5"

            
libs.db_connection.select_data_dict_with_commit(sql1, [])

In [ ]:
# Интересуют события у которых категория < 10
sql1 = "explain (analyze) select * from block11 where category < 10"

            
libs.db_connection.select_data_dict_with_commit(sql1, [])

In [ ]:
# Создаем индекс
sql1 = "create index block1_user on block11 (user_id)"

            
libs.db_connection.execute(sql1)

In [ ]:
# Как стало
sql1 = "explain (analyze) select * from block11 where user_id = 5"

            
libs.db_connection.select_data_dict_with_commit(sql1, [])

In [ ]:
# А что с категориями ?
sql1 = "explain (analyze) select * from block11 where category < 10"

            
libs.db_connection.select_data_dict_with_commit(sql1, [])

In [ ]:
# создаем индекс для категорий

sql1 = "create index block1_category on block11 (category)"

            
libs.db_connection.execute(sql1)

In [ ]:
# Как теперь работает запрос по категориям

sql1 = "explain (analyze) select * from block11 where category < 10"

            
libs.db_connection.select_data_dict_with_commit(sql1, [])

In [ ]:
# Удаляем индекс для категории

sql1 = "drop index block1_category"
libs.db_connection.execute(sql1)

In [ ]:
# Составной индекс (Multicolumn index)

sql1 = "explain (analyze) select * from block11 where category < 10 and type = 'click'"
            
libs.db_connection.select_data_dict_with_commit(sql1, [])

In [ ]:
# Создаем составной индекс для группы и типа события

sql1 = "create index block1_category_type on block11 (category, type)"
            
libs.db_connection.execute(sql1)

In [ ]:
# Как теперь работает запрос с фильтром по 2 полям

sql1 = "explain (analyze) select * from block11 where category < 10 and type = 'click'"

            
libs.db_connection.select_data_dict_with_commit(sql1, [])

In [ ]:
# Удаляем составной индекс

sql1 = "drop index block1_category_type"
libs.db_connection.execute(sql1)

In [ ]:
# Плохая селективность (часть 1)

sql1 = "explain (analyze) select type, count(1) from block11 group by type"
libs.db_connection.select_data_dict_with_commit(sql1, [])


In [ ]:
# Плохая селективность (часть 2)
sql1 = "create index block1_type on block11 (type)"
libs.db_connection.execute(sql1)

In [ ]:
# Плохая селективность (часть 3)

sql1 = "explain (analyze) select type, count(1) from block11 group by type"
libs.db_connection.select_data_dict_with_commit(sql1, [])

In [ ]:
# Задание: Нам нужно загрузить таблицу из источника. 
# При этом забирать ее будем инкрементально каждый день за прошлые сутки 
# селект уже есть мы его попробовали, но он что-то тормозит, нужно посмотреть, можно ли его ускорить

sql = "explain (analyze) select * from task1 where dt = '2022-05-19 00:00:00'"
libs.db_connection.select_data_dict_with_commit(sql, [])




In [ ]:
sql_index = "..."
libs.db_connection.execute(sql_index)

In [ ]:
# Как сейчас стало

sql = "explain (analyze) select * from task1 where dt = '2022-05-19 00:00:00'"
libs.db_connection.select_data_dict_with_commit(sql, [])